In [33]:
import pandas as pd
import pickle

https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2016-zip-code-data-soi

In [37]:
zc_df = pd.read_csv("./16zpallagi.csv")
# pop data: https://blog-splitwise-com.cdn.ampproject.org/v/s/blog.splitwise.com/2013/09/18/the-2010-us-census-population-by-zip-code-totally-free/amp/?usqp=mq331AQEKAFwAQ%3D%3D&amp_js_v=0.1#aoh=15686713597429&referrer=https%3A%2F%2Fwww.google.com&amp_tf=From%20%251%24s&ampshare=https%3A%2F%2Fblog.splitwise.com%2F2013%2F09%2F18%2Fthe-2010-us-census-population-by-zip-code-totally-free%2F
pop_df = pd.read_csv('./2010_census_zip_pop.csv')

* A02650: Total income
* NUMDEP: Number of Dependents
* N02300: Number of returns with unemployment

In [43]:
zc_df[["zipcode","NUMDEP"]]

,zipcode,NUMDEP
0,0,491310
1,0,360480
2,0,182880
3,0,130160
4,0,195990
5,0,56220
6,35004,660
7,35004,900
8,35004,660
9,35004,560


In [16]:
tax_df = zc_df.groupby('zipcode').mean().reset_index()[['zipcode','A02650', 'NUMDEP', 'N02300']]\
.rename(columns={'A02650': "total_income", 'NUMDEP': 'num_dependents', 'N02300': 'num_unemployed'})

tax_df.head()

,zipcode,total_income,num_dependents,num_unemployed
0,0,3.397431e+07,304947.581699,18300.490196
1,1001,8.542433e+04,651.666667,81.666667
2,1002,1.363077e+05,688.333333,48.333333
3,1003,4.948000e+02,0.000000,0.000000
4,1005,2.326833e+04,206.666667,31.666667


In [19]:
pop_df.rename(columns={'Zip Code ZCTA': 'zipcode', '2010 Census Population': '2010_population'}, inplace=True)

In [23]:
z_df = pop_df.merge(tax_df, on="zipcode", how='inner')

In [25]:
def clean_zip(z):
    cleaned = str(z) if z > 10000 else "0" + str(z)
    return cleaned

z_df.zipcode = z_df.zipcode.apply(clean_zip)

In [34]:
with open('./cleaned_original.pkl', 'rb') as file:
    cleaned_original = pickle.load(file)
cleaned_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 811 entries, 0 to 810
Data columns (total 6 columns):
building_rating_id    810 non-null float64
building_name         811 non-null object
address               811 non-null object
city                  811 non-null object
state                 811 non-null object
zipcode               811 non-null object
dtypes: float64(1), object(5)
memory usage: 38.1+ KB


In [36]:
merged_zip = cleaned_original.merge(z_df, on='zipcode', how='inner')

with open('./final_data.pkl', 'wb') as file:
    pickle.dump(merged_zip, file)